## Simulation Study

In [ ]:
# Empty the workspace
rm(list = ls())

# Set Folder-Path
path <- "~/LocalPredictability" # Change to project folder

In [ ]:
# Packages
library("tidyverse")
library("purrr")
library("magrittr")
library("glue")
library("rlang")
library("readr")
library("doParallel")
library("pbapply")
library("ggplot2")
library("cowplot")
library("gridExtra")
library("MASS")
library("glmnet")
library("rpart")
library("ranger")
library("lightgbm")
library("xgboost")
#install.packages(paste0(path, "/eDMA_1.5-3.tar.gz"), repos = NULL, type = "source")
library("eDMA") # Local Version
library("hdflex")
library("kableExtra")

In [ ]:
# Load Custom Functions
source(paste0(path, "/Code/_fmodels.R"))
source(paste0(path, "/Code/_cmodels.R"))
source(paste0(path, "/Code/_helpers.R"))
source(paste0(path, "/Code/_sim.R"))

# Convert Jupyter Notebook to R script
convert_ipynb_to_r("/Users/slehmann/Library/CloudStorage/Dropbox/HFTP/Code/simulation.ipynb")

---

## 1) Data-Generating Processes

In [7]:
### ------------------------------------------------
### Definition of Data-Generating Processes
# F1 -> Time-varying Parameter
# F2 -> Friedman Additive-Non-Linear
# F3 -> Friedman Interactive-Non-Linear
# F4 -> Additive-Linear (Dense)
# F0 -> Pure Noise

### Setting: Ultra-Sparse (TVP)
# Global Predictability + Abrupt Change
dgp1 <- list(
  list(start =   1, end = 120, func = c("F1"), tvp = c( 0.5, 0.0, 1.0), dense = NULL),
  list(start = 121, end = 320, func = c("F1"), tvp = c( 0.5, 0.0, 1.0), dense = NULL),
  list(start = 321, end = 569, func = c("F1"), tvp = c(-0.4, 0.0, 1.0), dense = NULL),
  list(start = 570, end = 620, func = c("F1"), tvp = c( 0.5, 0.0, 1.0), dense = NULL)
)

# Global Predictability + Gradual Change
dgp2 <- list(
  list(start =   1, end = 120, func = c("F1"), tvp = c( 0.8, -0.5, 420.0), dense = NULL),
  list(start = 121, end = 539, func = c("F1"), tvp = c( 0.8, -0.5, 420.0), dense = NULL),
  list(start = 540, end = 620, func = c("F1"), tvp = c(-0.2,  0.5, 420.0), dense = NULL)
)

# Local Predictability + Gradual Change
dgp3 <- list(
  list(start =  1,  end = 120, func = c("F1"), tvp = c( 0.0, -0.002, 1.0), dense = NULL),
  list(start = 121, end = 370, func = c("F1"), tvp = c( 0.0, -0.002, 1.0), dense = NULL),
  list(start = 371, end = 529, func = c("F0"), tvp = c(NULL, NULL, NULL),  dense = NULL),
  list(start = 530, end = 620, func = c("F1"), tvp = c( 0.0, -0.002, 1.0), dense = NULL)
)

# Local Predictability + Abrupt / Gradual Change
dgp4 <- list(
  list(start =   1, end = 120, func = c("F0"), tvp = c(NULL, NULL, NULL),  dense = NULL),
  list(start = 121, end = 180, func = c("F0"), tvp = c(NULL, NULL, NULL),  dense = NULL),
  list(start = 181, end = 299, func = c("F1"), tvp = c( 0.6, -0.2, 180.0), dense = NULL),
  list(start = 300, end = 370, func = c("F0"), tvp = c(NULL, NULL, NULL),  dense = NULL),
  list(start = 371, end = 539, func = c("F1"), tvp = c( 0.6,  0.2, 420.0), dense = NULL),
  list(start = 540, end = 620, func = c("F0"), tvp = c(NULL, NULL, NULL),  dense = NULL)
)
### ------------------------------------------------

### ------------------------------------------------
### Setting: Non-Linear, dense, sparse and structural breaks
# Global Friedman
dgp5 <- list(
  list(start =  1,  end = 120, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 620, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL)
)
dgp6 <- list(
  list(start =  1, end =  120, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 620, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL)
)

# Local Friedman
dgp7 <- list(
  list(start =   1, end = 120, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 210, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 211, end = 285, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 286, end = 375, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 376, end = 450, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 451, end = 540, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 541, end = 620, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL)
)
dgp8 <- list(
  list(start =   1, end = 120, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 210, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 211, end = 285, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 286, end = 375, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 376, end = 450, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 451, end = 540, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 541, end = 620, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL)
)

# Local Friedman + Local Linear-Additive (short noise periods)
dgp9 <- list(
  list(start =   1, end = 120, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 220, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 221, end = 250, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 251, end = 350, func = c("F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 351, end = 380, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 381, end = 480, func = c("F2"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 481, end = 510, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 511, end = 620, func = c("F4"), tvp = c(NULL, NULL, NULL), dense = 0.04)
)
dgp10 <- list(
  list(start =   1, end = 120, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 220, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 221, end = 250, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 251, end = 350, func = c("F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 351, end = 380, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 381, end = 480, func = c("F3"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 481, end = 510, func = c("F0"), tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 511, end = 620, func = c("F4"), tvp = c(NULL, NULL, NULL), dense = 0.04)
)

# Local Friedman + Local Linear-Additive (long noise periods)
dgp11 <- list(
  list(start =   1, end = 120, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 210, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 211, end = 260, func = c("F2", "F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 261, end = 350, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 351, end = 400, func = c("F2", "F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 401, end = 490, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 491, end = 540, func = c("F2", "F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 541, end = 620, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL)
)
dgp12 <- list(
  list(start =   1, end = 120, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 121, end = 210, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 211, end = 260, func = c("F3", "F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 261, end = 350, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 351, end = 400, func = c("F3", "F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 401, end = 490, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL),
  list(start = 491, end = 540, func = c("F3", "F4"), tvp = c(NULL, NULL, NULL), dense = 0.04),
  list(start = 541, end = 620, func = c("F0"),       tvp = c(NULL, NULL, NULL), dense = NULL)
)

### Combine in List
list_dgp <- list(dgp1, dgp2, dgp3, dgp4, dgp5, dgp6, dgp7, dgp8, dgp9, dgp10, dgp11, dgp12)
### ------------------------------------------------

In [ ]:
### ------------------------------------------------
### Visualization of DGPs
# Define common aesthetics
common_aes <- aes(x = start, xend = end, y = func, yend = func)
common_theme <- theme_bw() + theme(legend.position = "none")

# Plot each DGP
plot_list <- purrr::map(seq_along(list_dgp), ~{
  tibble(x = list_dgp[[.x]]) %>%
    unnest_wider(x) %>%
    dplyr::select(start, end, func) %>%
    dplyr::slice(-1) %>%
    mutate(start = start - 120, end = end - 120) %>%
    unnest_longer(func) %>%
    mutate(func = as.numeric(substr(func, 2, nchar(func)))) %>%
    ggplot(common_aes) +
    geom_segment(aes(color = ifelse(func == 0, "Noise", "Active")),
                 linewidth = 1.5) +
    scale_color_manual(values = c("Noise" = "grey", "Active" = "blue")) +
    geom_vline(xintercept = 120,
               linetype = "dashed",
               color = "red") +
    labs(x = "Time", y = "Active Component", title = paste("DGP", .x)) +
    scale_y_continuous(limit = c(0, 4),
                       breaks = c(0, 1, 2, 3, 4),
                       labels = c("Noise", "TVC", "Friedman1", "Friedman2", "Dense")) +
    common_theme
})

# Arrange the plots in a 6x2 grid
options(repr.plot.width = 16, repr.plot.height = 15)
gridExtra::grid.arrange(grobs = plot_list, ncol = 2, nrow = 6)
### ------------------------------------------------

## 2) Forecasting Accuracy

In [ ]:
######### --------------------------------------------
### Simulation Study -- Forecasting Accuracy ###
# Set Global Parameter
runs <- 100
n_init <- 120
n_obs <- 500
n_sig <- 500
rho <- 0.7
list_dgp <- list(dgp1, dgp2, dgp3, dgp4, dgp5, dgp6, dgp7, dgp8, dgp9, dgp10, dgp11, dgp12)
vec_snr <- 2.0
window_size <- 120

# Set up Parallel-Backend
cores <- 4
cl <- parallel::makeCluster(cores)
parallel::clusterExport(cl = cl,
                        varlist = c(
                          "path", "runs", "n_init", "n_obs", "n_sig",
                          "rho", "list_dgp", "vec_snr", "window_size"
                        ),
                        envir = environment())
parallel::clusterEvalQ(cl, {
  library("magrittr")
  library("glue")
  library("rlang")
  library("readr")
  library("doParallel")
  library("purrr")
  library("dplyr")
  library("tidyr")
  library("MASS")
  library("glmnet")
  library("rpart")
  library("ranger")
  library("xgboost")
  library("lightgbm")
  library("eDMA") #local version
  library("hdflex")
})
pbo <- pbapply::pboptions(type = "timer")

######### --------------------------------------------
### Parallel loop over reps
res <- pbapply::pblapply(cl = cl, X = seq_len(runs), FUN = function(r) {

  # Load Custom Functions
  source(glue::glue("{path}/Code/_fmodels.R"), local = TRUE)
  source(glue::glue("{path}/Code/_cmodels.R"), local = TRUE)
  source(glue::glue("{path}/Code/_sim.R"), local = TRUE)

  ######### --------------------------------------------
  ### Build Global Result Matrices
  # Benchmark Methods
  benchmark_names <- c("PHM", "STPHM",
                       "DART", "ReLa", "TRF",
                       "XGB", "TCSR",
                       #---#
                       "STSC", "STSC_EXF",
                       "STSCS10", "STSCSFLEX",
                       "PC5DMA", "PC10DMA", "PC15DMA")

  # Result Dimensions
  n_settings <- length(list_dgp) * length(vec_snr)
  n_models <- length(benchmark_names)

  # Setting-Names
  setting_names <- expand.grid(vec_snr, seq_len(length(list_dgp))) %>%
    purrr::pmap_chr(~ paste0("DGP", ..2, "_SNR", ..1))

  # Result-Object: Squared-Errors
  m <- matrix(NA, nrow = n_obs, ncol = length(benchmark_names),
              dimnames = list(seq_len(n_obs), benchmark_names))
  se <- setNames(base::replicate(n_settings, m, simplify = FALSE),
                 setting_names)

  # Result-Object: Predictions
  preds <- setNames(base::replicate(n_settings, m, simplify = FALSE),
                    setting_names)

  # Result-Object: Continuous-Ranked-Probability-Score
  crps_score <- setNames(base::replicate(n_settings, m, simplify = FALSE),
                         setting_names)

  # Result-Object: Mean-Squared-Error
  mse <- matrix(NA, nrow = n_settings, ncol = n_models,
                dimnames = list(setting_names, benchmark_names))

  # Result-Object: Mean-Continuous-Ranked-Probability-Score
  mcrps <- matrix(NA, nrow = n_settings, ncol = n_models,
                  dimnames = list(setting_names, benchmark_names))

  # Result-Object: STSC-Parameters
  v <- rep(NA, n_obs)
  m1 <- matrix(NA, nrow = n_obs, ncol = (n_sig + 12),
               dimnames = list(seq_len(n_obs), NULL))
  m2 <- matrix(NA, nrow = n_obs, ncol = 3,
               dimnames = list(seq_len(n_obs), NULL))
  params <- setNames(base::replicate(n_settings,
                                     list(setting = NA,
                                          response = v,
                                          gamma = v,
                                          psi = v,
                                          signals = m1,
                                          lambda = m2,
                                          kappa = m2),
                                     simplify = FALSE), setting_names)

  # Remove & Clean Memory
  rm(list = c("m", "m1", "m2", "v"))

  ######### --------------------------------------------
  # Init Counter
  i <- 1

  # Loop over settings
  for (dgp in list_dgp) {
    for (snr in vec_snr) {

      ### Save Parameter ###
      params[[i]][[1]] <- tidyr::tibble(x = dgp) %>%
        tidyr::unnest_wider(x) %>%
        dplyr::select(start, end, func) %>%
        dplyr::slice(-1) %>%
        dplyr::mutate(start = start - n_init) %>%
        dplyr::mutate(end = end - n_init) %>%
        dplyr::mutate(snr = snr)

      ### Data ###
      # Simulate Data
      dataset <- sim_data(n_obs,
                          n_init,
                          n_sig,
                          dgp,
                          rho,
                          snr,
                          r)

      # Set Response and Signals
      y_init <- dataset$response
      X_init <- dataset$covariates

      # Get Active Signals
      vec_active <- dataset$active

      # Get (random) Noise-Signals
      vec_nactive <- dataset$non_active

      # Remove
      rm(list = c("dataset"))

      ####### ------------------------------------------------
      ### Part 1: Create F-Signals
      # Tuning Parameter - Regression Trees
      dt_depth <- c(1, 2, 3, 4)
      dt_windows <- c(60, 120)

      # Tuning Parameter - Ridge / Elastic Net
      eln_alpha <- c(0.0, 0.50)
      eln_windows <- c(60, 120)

      # Time Sequence
      start <- n_init # max(dt_windows, eln_windows)
      end <- n_obs + n_init - 1
      t_seq <- seq(start, end)

      # Result-Object: F-Signals
      coln_dt <- apply(expand.grid(dt_depth, dt_windows), 1,
                       function(x) paste0("TREE_W", x[2], "_D", x[1]))
      coln_eln <- apply(expand.grid(eln_alpha, eln_windows), 1,
                        function(x) paste0("ELN_W", x[2], "_A", x[1]))
      Ext_F_init <- matrix(NA, ncol = length(c(coln_dt, coln_eln)),
                           nrow = (n_obs + n_init),
                           dimnames = list(seq_len(n_obs + n_init),
                                           c(coln_dt, coln_eln)))

      # Loop over time
      for (t in t_seq) {

        ### Split Data ###
        # Train Data
        x_train <- X_init[1:t, , drop = FALSE]
        y_train <- y_init[1:t, , drop = FALSE]

        # Predict Data
        x_pred <- X_init[t + 1, , drop = FALSE]
        y_pred <- y_init[t + 1, ]

        ### Decision Trees
        # Fit & Predict Model
        dt_idx <- seq_along(coln_dt)
        Ext_F_init[t + 1, dt_idx] <- cm_dt(x_train,
                                           y_train,
                                           x_pred,
                                           dt_depth,
                                           c(vec_active, vec_nactive),
                                           dt_windows)

        ### Elastic Net
        # Fit & Predict Model
        eln_idx <- length(coln_dt) + seq_along(coln_eln)
        Ext_F_init[t + 1, eln_idx] <- cm_eln(x_train,
                                             y_train,
                                             x_pred,
                                             eln_alpha,
                                             c(vec_active, vec_nactive),
                                             eln_windows,
                                             t)
      }

      ### Remove Init-Period
      y <- y_init[t_seq + 1, , drop = FALSE]
      X <- X_init[t_seq + 1, , drop = FALSE]
      Ext_F <- Ext_F_init[t_seq + 1, , drop = FALSE]

      # Save Response-Variable
      params[[i]]$response <- y

      # Remove & Clean Memory
      rm(list = c("y_init", "X_init", "Ext_F_init", "x_train", "y_train",
                  "x_pred", "y_pred", "dt_idx", "eln_idx", "start", "end",
                  "t_seq", "coln_dt", "coln_eln"))
      invisible(gc())

      # Check for Missing Values
      if (any(is.na(y)) || any(is.na(X)) || any(is.na(Ext_F))) {
        rlang::abort("Error: Missing Values in Data")
      }

      # Check Length
      if (any(c(nrow(y), nrow(X), nrow(Ext_F)) != n_obs)) {
        rlang::abort("Error: Length of Data is not correct")
      }

      ####### ------------------------------------------------
      ### Part 2: Benchmark ###
      ### Prevailing Historical Mean (PHM)
      # Apply Function
      phm_results <- phm(y, window_size, "expanding")

      # Assign Results
      preds[[i]][, "PHM"] <- phm_results[[1]]
      se[[i]][, "PHM"] <- (y - phm_results[[1]]) ** 2

      ### Density Transformation (ST-PHM)
      # Apply Function
      stphm_results <- hdflex::tvc(y,
                                   NULL,
                                   preds[[i]][, "PHM", drop = FALSE],
                                   5 * 12,
                                   1.00,
                                   0.97,
                                   FALSE)

      # Assign Results
      preds[[i]][, "STPHM"] <- stphm_results$Forecasts$Point_Forecasts
      se[[i]][, "STPHM"] <- (y - stphm_results$Forecasts$Point_Forecasts) ** 2
      crps_score[[i]][, "STPHM"] <- crps(y, stphm_results$Forecasts$Point_Forecasts,
                                         sqrt(stphm_results$Forecasts$Variance_Forecasts),
                                         NULL, "normal")$crps

      # Remove & Clean Memory
      rm(list = c("phm_results", "stphm_results"))
      invisible(gc())

      ####### ------------------------------------------------
      ### Part 3: Forecasting Models with Rolling Window ###
      # Time Sequence
      t_start <- window_size  # -> change here if window_size != OOS-Start-Date
      t_end <- n_obs - 1
      t_seq <- seq(t_start, t_end)

      # Window Check
      if (t_start < window_size) {
        rlang::abort("Error: t_seq misspecified")
      }

      # Combine P- and F-Signals
      S <- cbind(X, Ext_F)

      # Loop with Rolling Window
      for (t in t_seq) {

        ### Split Data ###
        # Train Data
        s_train <- S[(t - window_size + 1):t, , drop = FALSE]
        y_train <- y[(t - window_size + 1):t, , drop = FALSE]

        # Predict Data
        s_pred <- S[t + 1, , drop = FALSE]
        y_pred <- y[t + 1, ]

        ####### Model 1: Relaxed-Lasso #######
        # Parameter
        rela_folds <- 5

        # Fit and Predict
        pred_relax <- relasso_model(s_train,
                                    y_train,
                                    s_pred,
                                    rela_folds,
                                    t)

        ####### Model 2: XGBoost #######
        # Parameter
        xgb_folds  <- 5
        xgb_ntrees <- 500
        xgb_lr     <- 0.1
        xgb_target <- NULL
        xgb_cores  <- 1

        # Predict
        pred_xgb <- xgb_model(s_train,
                              y_train,
                              s_pred,
                              xgb_folds,
                              xgb_ntrees,
                              xgb_lr,
                              xgb_target,
                              xgb_cores,
                              t)

        ####### Model 3: LightGBM #######
        # Parameter
        dart_folds  <- 5
        dart_ntrees <- 500
        dart_lr     <- 0.1
        dart_dr     <- 0.1
        dart_target <- NULL
        dart_cores  <- 1

        # Fit and Predict
        pred_dart <- dart_model(s_train,
                                y_train,
                                s_pred,
                                dart_folds,
                                dart_ntrees,
                                dart_lr,
                                dart_dr,
                                dart_target,
                                dart_cores,
                                t)

        ####### Model 4: (Targeted) RandomForests #######
        # Parameter
        trf_n_target  <- 50
        trf_ntrees    <- 500
        trf_max_depth <- 3
        trf_cores     <- 1

        # Fit and Predict
        pred_trf <- trf_model(s_train,
                              y_train,
                              s_pred,
                              trf_n_target,
                              trf_ntrees,
                              trf_max_depth,
                              trf_cores,
                              t)

        ####### Model 5: (Targeted) Complete Subset Regression #######
        # Parameter
        csr_n_target <- 20
        csr_n_subset <- 10
        csr_ubound   <- 10000
        csr_sampling <- TRUE

        # Fit and Predict
        pred_tcsr <- csr_model(s_train,
                               y_train,
                               s_pred,
                               csr_n_target,
                               csr_n_subset,
                               csr_ubound,
                               csr_sampling)

        ####### Save Predictions #######
        preds[[i]][t + 1, "ReLa"] <- pred_relax
        preds[[i]][t + 1, "XGB"]  <- pred_xgb
        preds[[i]][t + 1, "DART"] <- pred_dart
        preds[[i]][t + 1, "TRF"]  <- pred_trf
        preds[[i]][t + 1, "TCSR"] <- pred_tcsr

        ####### Save Squared Errors #######
        se[[i]][t + 1, "ReLa"] <- (y_pred - pred_relax) ** 2
        se[[i]][t + 1, "XGB"]  <- (y_pred - pred_xgb)   ** 2
        se[[i]][t + 1, "DART"] <- (y_pred - pred_dart)  ** 2
        se[[i]][t + 1, "TRF"]  <- (y_pred - pred_trf)   ** 2
        se[[i]][t + 1, "TCSR"] <- (y_pred - pred_tcsr)  ** 2
      }

      # Remove & Clean Memory
      rm(list = c("S", "s_train", "y_train", "s_pred", "y_pred",
                  "pred_relax", "pred_xgb", "pred_dart",
                  "pred_trf", "pred_tcsr"))
      invisible(gc())

      ####### ------------------------------------------------
      ### Part 2: STSC and Variants ###
      ####### STSC-Model #######
      # Set TV-C-Parameter
      init <- 5 * 12
      lambda_grid <- c(0.9667, 0.9833, 1.0000)
      kappa_grid <- c(0.93, 0.95, 0.97)
      bias <- TRUE

      # Set DSC-Parameter
      gamma_grid <- c(0.40, 0.50, 0.60, 0.70, 0.80, 0.90,
                      0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00)
      n_tvc <- (ncol(X) + ncol(Ext_F)) * length(lambda_grid) * length(kappa_grid)
      psi_grid <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc / 4)))
      delta <- 0.9833
      burn_in <- init / 2
      burn_in_dsc <- 1
      metric <- 5
      equal_weight <- TRUE
      incl <- NULL
      parallel <- FALSE
      n_threads <- NULL

      # Apply STSC-Function
      stsc_results <- hdflex::stsc(y,
                                   X,
                                   Ext_F,
                                   init,
                                   lambda_grid,
                                   kappa_grid,
                                   bias,
                                   gamma_grid,
                                   psi_grid,
                                   delta,
                                   burn_in,
                                   burn_in_dsc,
                                   metric,
                                   equal_weight,
                                   incl,
                                   parallel,
                                   n_threads,
                                   NULL)

      # Assign Results
      pred_stsc <- stsc_results$Forecasts$Point_Forecasts
      var_stsc  <- stsc_results$Forecasts$Variance_Forecasts

      # Save Parameter
      params[[i]]$gamma <- stsc_results$Tuning_Parameters$Gamma
      params[[i]]$psi <- stsc_results$Tuning_Parameters$Psi
      params[[i]]$signals <- stsc_results$Tuning_Parameters$Signals
      params[[i]]$lambda <- stsc_results$Tuning_Parameters$Lambda
      params[[i]]$kappa <- stsc_results$Tuning_Parameters$Kappa

      # Remove & Clean Memory
      rm(list = c("stsc_results"))

      ####### STSC - Excluding-F-Signals #######
      n_tvc_exf <- (ncol(X) + 0) * length(lambda_grid) * length(kappa_grid)
      psi_grid_exf <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc_exf / 4)))

      # Apply STSC-Function
      stsc_exf_results <- hdflex::stsc(y,
                                       X,
                                       NULL,
                                       init,
                                       lambda_grid,
                                       kappa_grid,
                                       bias,
                                       gamma_grid,
                                       psi_grid_exf,
                                       delta,
                                       burn_in,
                                       burn_in_dsc,
                                       metric,
                                       equal_weight,
                                       incl,
                                       parallel,
                                       n_threads,
                                       NULL)

      # Assign Results
      pred_stsc_exf <- stsc_exf_results$Forecasts$Point_Forecasts
      var_stsc_exf  <- stsc_exf_results$Forecasts$Variance_Forecasts

      # Remove & Clean Memory
      rm(list = c("stsc_exf_results"))

      ####### STSC - Selecting 10 Forecasts with LASSO #######
      # Set TV-C-Parameter
      init <- 5 * 12
      lambda_grid <- c(0.9667, 0.9833, 1.0000)
      kappa_grid <- 0.97
      bias <- TRUE
      n_cores <- 1

      # Set Lasso-Target-Parameter
      stscs10_target <- 10

      # Apply STSC-S-X Function
      stscs10_results <- stscsx(y,
                                X,
                                Ext_F,
                                lambda_grid,
                                kappa_grid,
                                init,
                                bias,
                                n_cores,
                                window_size,
                                stscs10_target)

      # Assign Results
      pred_stscs10 <- stscs10_results[, 1]
      var_stscs10  <- stscs10_results[, 2]

      # Remove & Clean Memory
      rm(list = c("stscs10_results"))

      ####### STSC - Using LASSO to dynamically select forecasts #######
      # Set TV-C-Parameter
      init <- 5 * 12
      lambda_grid <- c(0.9667, 0.9833, 1.0000)
      kappa_grid <- 0.97
      bias <- TRUE
      n_cores <- 1

      # Set Lasso-CV-Parameter
      stscsflex_folds <- 5

      # Apply ST-S-FLEX-Function
      stscsflex_results <- stscsflex(y,
                                     X,
                                     Ext_F,
                                     lambda_grid,
                                     kappa_grid,
                                     init,
                                     bias,
                                     n_cores,
                                     window_size,
                                     stscsflex_folds)

      # Assign Results
      pred_stscsflex <- stscsflex_results[, 1]
      var_stscsflex  <- stscsflex_results[, 2]

      # Remove & Clean Memory
      rm(list = c("stscsflex_results"))

      ####### (Principal Component) Dynamic Model Averaging #######
      # Set Parameter
      pcdma_win <- window_size
      pcdma_comp <- 5
      pcdma_alpha <- 0.99
      pcdma_lambda <- c(0.9667, 0.9833, 1.0000)
      pcdma_kappa <- 0.97
      pcdma_cores <- 1
      pcdma_exl <- NULL

      # Apply PCA-DMA
      pc5dma_results <- pcdma(y,
                              X,
                              Ext_F,
                              pcdma_win,
                              pcdma_comp,
                              pcdma_alpha,
                              pcdma_lambda,
                              pcdma_kappa,
                              pcdma_cores,
                              pcdma_exl)

      # Assign Results
      pred_pc5dma <- pc5dma_results[, 1]
      var_pc5dma  <- pc5dma_results[, 3]
      crps_pc5dma <- pc5dma_results[, 4]

      # Remove & Clean Memory
      rm(list = c("pc5dma_results"))

      # Set Parameter
      pcdma_comp <- 10

      # Apply PCA-DMA
      pc10dma_results <- pcdma(y,
                               X,
                               Ext_F,
                               pcdma_win,
                               pcdma_comp,
                               pcdma_alpha,
                               pcdma_lambda,
                               pcdma_kappa,
                               pcdma_cores,
                               pcdma_exl)

      # Assign Results
      pred_pc10dma <- pc10dma_results[, 1]
      var_pc10dma  <- pc10dma_results[, 3]
      crps_pc10dma <- pc10dma_results[, 4]

      # Remove & Clean Memory
      rm(list = c("pc10dma_results"))

      # Set Parameter
      pcdma_comp <- 15

      # Apply PCA-DMA
      pc15dma_results <- pcdma(y,
                               X,
                               Ext_F,
                               pcdma_win,
                               pcdma_comp,
                               pcdma_alpha,
                               pcdma_lambda,
                               pcdma_kappa,
                               pcdma_cores,
                               pcdma_exl)

      # Assign Results
      pred_pc15dma <- pc15dma_results[, 1]
      var_pc15dma  <- pc15dma_results[, 3]
      crps_pc15dma <- pc15dma_results[, 4]

      # Remove & Clean Memory
      rm(list = c("pc15dma_results"))

      ### Check Dimensions
      if (any(c(length(pred_stsc), length(pred_stsc_exf),
                length(pred_stscs10), length(pred_stscsflex),
                length(pred_pc5dma), length(pred_pc10dma), length(pred_pc15dma))
              != n_obs)) {
        rlang::abort("Error: Length of Predictions is not correct")
      }

      ####### Save Point Forecasts #######
      preds[[i]][, "STSC"] <- pred_stsc
      preds[[i]][, "STSC_EXF"] <- pred_stsc_exf
      preds[[i]][, "STSCS10"] <- pred_stscs10
      preds[[i]][, "STSCSFLEX"] <- pred_stscsflex
      preds[[i]][, "PC5DMA"] <- pred_pc5dma
      preds[[i]][, "PC10DMA"] <- pred_pc10dma
      preds[[i]][, "PC15DMA"] <- pred_pc15dma

      ####### Save Squared Errors #######
      se[[i]][, "STSC"] <- (y - pred_stsc) ** 2
      se[[i]][, "STSC_EXF"] <- (y - pred_stsc_exf) ** 2
      se[[i]][, "STSCS10"] <- (y - pred_stscs10) ** 2
      se[[i]][, "STSCSFLEX"] <- (y - pred_stscsflex) ** 2
      se[[i]][, "PC5DMA"] <- (y - pred_pc5dma) ** 2
      se[[i]][, "PC10DMA"] <- (y - pred_pc10dma) ** 2
      se[[i]][, "PC15DMA"] <- (y - pred_pc15dma) ** 2

      ##### Save Continuous Ranked Probability Score #######
      crps_score[[i]][, "STSC"] <- crps(y, pred_stsc, sqrt(var_stsc), NULL, "normal")$crps # nolint
      crps_score[[i]][, "STSC_EXF"] <- crps(y, pred_stsc_exf, sqrt(var_stsc_exf), NULL, "normal")$crps # nolint
      crps_score[[i]][, "STSCS10"] <- crps(y, pred_stscs10, sqrt(var_stscs10), NULL, "normal")$crps # nolint
      crps_score[[i]][, "STSCSFLEX"] <- crps(y, pred_stscsflex, sqrt(var_stscsflex), NULL, "normal")$crps # nolint
      crps_score[[i]][, "PC5DMA"] <- crps_pc5dma
      crps_score[[i]][, "PC10DMA"] <- crps_pc10dma
      crps_score[[i]][, "PC15DMA"] <- crps_pc15dma

      ####### ------------------------------------------------
      ### Part 3: Evaluation ###
      ####### OOS-Period #######
      # Define Evaluation Period (OOS-Period)
      oos_idx <- t_seq + 1
      oos_y <- y[oos_idx, ]

      # Check Negative Values
      if (any(se[[i]][oos_idx, ] < 0)) {
        rlang::abort("Error: Negative Values in Squared Errors")
      }
      # Check for Missing Values
      if (any(is.na(se[[i]][oos_idx, ])) || any(is.na(preds[[i]][oos_idx, ]))) {
        rlang::abort("Error: Missing Values in Squared Errors | Predictions")
      }

      ####### Mean-Squared-Errors #######
      # Calculate MSE
      mse[i, ] <- colMeans(se[[i]][oos_idx, ])

      ####### Mean-CRPS #######
      # Calculate CRPS
      mcrps[i, ] <- colMeans(crps_score[[i]][oos_idx, ])
      ####### ------------------------------------------------

      # Remove & Clean Memory
      rm(list = c("oos_y", "y", "X", "Ext_F",
                  "pred_stphm",
                  "pred_stsc", "pred_stsc_exf",
                  "pred_stscs10", "pred_stscsflex",
                  "pred_pc5dma", "pred_pc10dma", "pred_pc15dma",
                  "var_stphm",
                  "var_stsc", "var_stsc_exf",
                  "var_stscs10", "var_stscsflex",
                  "var_pc5dma", "var_pc10dma", "var_pc15dma",
                  "oos_idx"))
      invisible(gc())

      # Update Counter
      i <- i + 1
    }
  }

  ####### ------------------------------------------------
  # Save Predictions
  readr::write_rds(preds,
                   glue::glue("{path}/Results/SIM/Accuracy/preds_{r}.rds"),
                   compress = "gz")

  # Save Squared-Errors
  readr::write_rds(se,
                   glue::glue("{path}/Results/SIM/Accuracy/se_{r}.rds"),
                   compress = "gz")

  # Save Continuous Ranked Probability Score
  readr::write_rds(crps_score,
                   glue::glue("{path}/Results/SIM/Accuracy/crps_{r}.rds"),
                   compress = "gz")

  # Save STSC-Parameter
  readr::write_rds(params,
                   glue::glue("{path}/Results/SIM/Accuracy/params_{r}.rds"),
                   compress = "gz")

  # Save Results
  readr::write_rds(list(mse = mse, mcrps = mcrps),
                   glue::glue("{path}/Results/SIM/Accuracy/res_{r}.rds"),
                   compress = "gz")

  # Return Results
  return(list(mse = mse, mcrps = mcrps))
})

# Stop Cluster
parallel::stopCluster(cl)

# Mean-Squared-Errors
res_mse <- Reduce("+", purrr::map(res, 1)) / runs
mse_scores <- res_mse / res_mse[, "STSC"]

# Continuous Ranked Probability Score
res_crps <- Reduce("+", purrr::map(res, 2)) / runs
crsps_scores <- res_crps / res_crps[, "STSC"]

# Show
t(mse_scores)
t(crsps_scores)
####### ------------------------------------------------

## 3.) Computational Runtime

In [ ]:
####### ------------------------------------------------
### Computational Run-Time
# Benchmark Names
benchmark_names <- c("DART", "ReLa", "XGB", "TRF", "TCSR",
                     #---#
                     "STSC",
                     "STSCS10", "STSCSFLEX",
                     "PC5DMA", "PC10DMA", "PC15DMA")

### Settings
n_obs <- c(500, 1000)
n_sigs <- c(500, 1000)
windows <- c(120, 620)
params <- cbind(n_obs = n_obs, n_sigs = n_sigs, win = windows)

### Result Matrix
m  <- matrix(NA, nrow = nrow(params), ncol = 2 + length(benchmark_names),
             dimnames = list(seq_len(nrow(params)), c("T", "K", benchmark_names)))

# Loop
for (i in seq_len(nrow(params))) {

  # Status
  print(i)

  # Assign Settings
  n_obs  <- params[i, 1]
  n_sigs <- params[i, 2]
  win    <- params[i, 3]

  # Simulate Data
  dataset <- sim_data_ct(n_obs, n_sigs)

  # Assign
  y <- dataset$y
  X <- dataset$X

  # Benchmark
  mbm <- microbenchmark::microbenchmark(

    benchmark_dart(y, X, win),
    benchmark_rela(y, X, win),
    benchmark_xgb(y, X, win),
    benchmark_trf(y, X, win),
    benchmark_tcsr(y, X, win),
    benchmark_stsc(y, X),
    benchmark_stscsx(y, X, win),
    benchmark_stscsflex(y, X, win),
    benchmark_pcdma(y, X, win, 5),
    benchmark_pcdma(y, X, win, 10),
    benchmark_pcdma(y, X, win, 15),

    unit  = "seconds",
    times = 5
  )

  # Assign Results
  m[i, ] <- cbind(n_obs, n_sigs, matrix(summary(mbm)$mean, nrow = 1))
}

# Save
write_rds(m, glue("{path}/Results/SIM/Timing/timing_results.rds"),
          compress = "gz")

# Absolute Results
round(m, 2)
####### ------------------------------------------------

## 4.) Plots

In [4]:
####### ------------------------------------------------
### Global Settings
black_color <- "#000000"
grey_color <- "#807f7f"
orange_color <- "#E69F00"
darkblue_color <- "#3a5795"
sky_color <- "#56B4E9"
green_color <- "#009E73"
blue_color <- "#0072B2"
red_color <- "#D55E00"
purple_color <- "#CC79A7"
dotted_line_color <- "#7b7979da"
lightgrey_color <- "#807f7f18"
text_size <- 24
axis_text_size <- 20
legend_text_size <- 20
legend_key_size <- unit(4, "line")
x_breaks <- seq(0, 500, 100)
x_expand <- expansion(mult = c(0, 0.025))
y_expand <- expansion(mult = c(0.1, 0.1))
init_idx <- c(1:120)
hline_size <- 0.5
vline_size <- 0.5

# Set the theme globally
theme_set(
  theme_bw() +
    theme(
      plot.title = element_text(size = text_size, face = "bold", hjust = 0.5, margin = margin(0, 0, 10, 0)),
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(),
      text = element_text(size = text_size),
      axis.text = element_text(size = axis_text_size),
      axis.text.x = element_text(vjust = 0.5),
      axis.title.x = element_text(vjust = -0.5),
      axis.title = element_text(size = text_size),
      strip.text = element_text(size = text_size),
      legend.title = element_blank(),
      legend.text = element_text(size = legend_text_size),
      legend.position = "bottom",
      legend.margin = margin(t = -30),
      legend.key.size = legend_key_size
    )
)

# Common Plot Asthetics
plot_aesthetics <- list(
  scale_x_continuous(breaks = x_breaks,
                     expand = x_expand),
  geom_vline(xintercept = 120,
             linetype = "dashed",
             color = red_color,
             linewidth = vline_size),
  geom_hline(yintercept = 0,
             linetype = "dotted",
             color = dotted_line_color,
             linewidth = hline_size)
)
####### ------------------------------------------------

In [ ]:
####### ------------------------------------------------
### Load and combine simulation runs
# SNR-Level
snr <- "SNR2"

# Squared-Error Files
files_se <- list.files(glue("{path}/Results/SIM/Accuracy/{snr}"),
                       full.names = TRUE,
                       pattern = "se_")

# CRPS Files
files_crps <- list.files(glue("{path}/Results/SIM/Accuracy/{snr}"),
                         full.names = TRUE,
                         pattern = "crps_")

# STSC-Parameter Files
files_params <- list.files(glue("{path}/Results/SIM/Accuracy/{snr}"),
                           full.names = TRUE,
                           pattern = "params_")

# Load Files
se          <- lapply(files_se, readRDS)
crps_scores <- lapply(files_crps, readRDS)
stsc_params <- lapply(files_params, readRDS)

# Check SNR
glue("SNR{purrr::map(stsc_params, 1)[[1]]$setting$snr[1]}")

# Get Number of Simulation runs
mc_runs <- length(se)
####### ------------------------------------------------

In [9]:
####### ------------------------------------------------
### Plot: DGP 1 - 4 (Appendix)
# Replicate Data for Time-Varying Coefficients DGPs
data_theta <- list(
  data.frame(xstart = c(1, 201, 450), xend = c(200, 449, 500), ystart = c(0.5, -0.4, 0.5), yend = c(0.5, -0.4, 0.5)),
  data.frame(xstart = c(1, 420), xend = c(419, 500), ystart = c(0.8, 0.3), yend = c(0.3, 0.4)),
  data.frame(xstart = c(1, 251, 410), xend = c(250, 409, 500), ystart = c(-0.002, 0, -0.82), yend = c(-0.5, 0, -1.0)),
  data.frame(xstart = c(1, 61, 180, 251, 420), xend = c(60, 179, 250, 419, 500), ystart = c(0, 0.53, 0.0, 0.72, 0.0), yend = c(0, 0.401, 0.0, 0.80, 0.0))
)

# Replicate Noise-Periods
shaded_areas <- list(
  data.frame(start = c(0), end = c(0)),
  data.frame(start = c(0), end = c(0)),
  data.frame(start = c(251), end = c(409)),
  data.frame(start = c(1, 180, 420), end = c(60, 250, 500))
)

# Loop over DGP
for (i in seq(4)) {

  # Calculate Average Values
  mean_se <- Reduce("+", purrr::map(se, i)) / mc_runs
  mean_crps <- Reduce("+", purrr::map(crps_scores, i)) / mc_runs

  # Mark OOS-Period
  mean_se[init_idx, ] <- 0
  mean_crps[init_idx, ] <- 0

  ### Plot: Time-varying Parameter
  # Prepare and plot theta
  plot_theta <- data_theta[[i]] %>%
    ggplot(aes(x = xstart, xend = xend, y = ystart, yend = yend)) +
    geom_segment(linewidth = 1.0, aes(color = "Time-varying coefficient")) +
    scale_color_manual(values = black_color) +
    labs(title = expression(bold(paste("Evolution of ", beta["1,t"]))),
         y = glue("DGP {i}"),
         x = "") +
    scale_y_continuous(labels = sprintf("%.1f", seq(-1, 1, 0.5)),
                       breaks = seq(-1, 1, 0.5),
                       limits = c(-1, 1),
                       expand = y_expand) +
    theme(legend.position = ifelse((i == 4), "bottom", "none")) +
    theme(plot.title = element_text(colour = ifelse(i == 1, black_color, "white"))) +
    guides(color = guide_legend(order = 1),
           size  = guide_legend(order = 2)) +
    plot_aesthetics

  ### Plot: Cumulative Squared Error Differences (CSSED)
  # Load Data
  df <- mean_se %>%
    as_tibble(rownames = "row") %>%
    mutate(row = as.numeric(row)) %>%
    mutate(across(-1, ~PHM - .)) %>%
    mutate(across(-1, cumsum)) %>%
    dplyr::select(-PHM, -STPHM) %>%
    pivot_longer(!row, names_to = "Model", values_to = "value") %>%
    mutate("Method" = ifelse(Model == "STSC", "STSC", "Benchmark Methods")) %>%
    mutate(Method = factor(Method, levels = c("STSC", "Benchmark Methods")))

  # Plot CSSED
  plot_cssed <- df %>%
    ggplot() +
    geom_line(aes(row, value, group = Model, color = Method, linetype = Method, linewidth = Method, alpha = Method)) +
    labs(title = "Cumulative Squared Error Differences", y = "", x = "") +
    scale_linetype_manual(values = c(1, 2)) +
    scale_linewidth_manual(values = c(0.75, 0.5), guide = "none") +
    scale_alpha_manual(values = c(1, 0.5), guide = "none")  +
    scale_color_manual(values = c(black_color, grey_color)) +
    scale_y_continuous(expand = y_expand) +
    theme(plot.title = element_text(colour = ifelse(i == 1, black_color, "white"))) +
    theme(legend.position = ifelse((i == 4), "bottom", "none")) +
    guides(color = guide_legend(override.aes = list(size = 14, alpha = 1, linewidth = 1.25))) +
    geom_rect(data = shaded_areas[[i]],
              aes(xmin = start,
                  xmax = end,
                  ymin = -Inf,
                  ymax = Inf),
              fill = lightgrey_color, #sky_color,
              color = black_color, #darkblue_color,
              show.legend = FALSE) +
    plot_aesthetics

  ### Plot: Cumulative CRPS Differences
  # Load Data
  df <- mean_crps %>%
    as_tibble(rownames = "row") %>%
    mutate(row = as.numeric(row)) %>%
    mutate(across(-1, ~STPHM - .)) %>%
    mutate(across(-1, cumsum)) %>%
    dplyr::select(-PHM, -STPHM, -DART, -ReLa, -TRF, -XGB, -TCSR) %>%
    pivot_longer(!row, names_to = "Model", values_to = "value") %>%
    mutate("Method" = ifelse(Model == "STSC", "STSC", "Benchmark Methods")) %>%
    mutate(Method = factor(Method, levels = c("STSC", "Benchmark Methods")))

  # Plot CRPS
  plot_crps <- df %>%
    ggplot() +
    geom_line(aes(row, value, group = Model, color = Method, linetype = Method, linewidth = Method, alpha = Method)) +
    labs(title = "Cumulative CRPS Differences", y = "", x = "") +
    scale_linetype_manual(values = c(1, 2)) +
    scale_linewidth_manual(values = c(0.75, 0.5), guide = "none") +
    scale_alpha_manual(values = c(1, 0.5), guide = "none")  +
    scale_color_manual(values = c(black_color, grey_color)) +
    scale_y_continuous(expand = y_expand) +
    theme(plot.title = element_text(colour = ifelse(i == 1, black_color, "white"))) +
    theme(legend.position = ifelse((i == 4), "bottom", "none")) +
    guides(color = guide_legend(override.aes = list(size = 14, alpha = 1, linewidth = 1.25))) +
    geom_rect(data = shaded_areas[[i]],
              aes(xmin = start,
                  xmax = end,
                  ymin = -Inf,
                  ymax = Inf),
              fill = lightgrey_color, #sky_color,
              color = black_color, #darkblue_color,
              show.legend = FALSE) +
    plot_aesthetics

  # Save
  write_rds(plot_theta, glue("{path}/Results/SIM/Plots/plot_theta_{i}.rds"))
  write_rds(plot_cssed, glue("{path}/Results/SIM/Plots/plot_cssed_{i}.rds"))
  write_rds(plot_crps, glue("{path}/Results/SIM/Plots/plot_crps_{i}.rds"))
}
####### ------------------------------------------------

In [10]:
####### ------------------------------------------------
### Plot: DGP 5 - 12 (main script)
# Loop over DGPs
for (i in 5:12) {

  # Calculate Average Values
  mean_se <- Reduce("+", purrr::map(se, i)) / mc_runs
  mean_crps <- Reduce("+", purrr::map(crps_scores, i)) / mc_runs

  # Mark OOS-Period
  mean_se[init_idx, ]   <- 0
  mean_crps[init_idx, ] <- 0

  ## Get Structural Breaks
  dgp_data <- purrr::map(stsc_params, i)[[1]]$setting

  # Get Noise Periods (-> Shaded Areas)
  shaded_areas <- dgp_data %>%
    dplyr::select(start, end, func) %>%
    filter(func == "F0")

  ### Plot: Active Components
  # Load and plot data
  plot_dgp <- dgp_data %>%
    unnest_longer(func) %>%
    mutate(func = as.numeric(substr(func, 2, nchar(func)))) %>%
    mutate(func = ifelse(func > 0, func - 1, 0)) %>%
    mutate(func = ifelse(func == 0, 4, func)) %>%
    ggplot(aes(x = start, xend = end, y = func, yend = func)) +
    geom_segment(aes(color = ifelse(func == -1, "Noise", "Active")),
                 linewidth = 1.00) +
    scale_color_manual(values = c("Noise" = grey_color, "Active" = black_color),
                       breaks = c("Noise", "Active"),
                       labels = c("Noise", "Active"),
                       drop = FALSE) +
    labs(x = "", y = glue::glue("DGP {i-4}"), title = "Active Function") +
    scale_y_continuous(limit = c(1, 4),
                       breaks = c(1, 2, 3, 4),
                       labels = c(expression(f[1]),
                                  expression(f[2]),
                                  expression(f[3]),
                                  expression(f[4])),
                       expand = y_expand) +
    theme(legend.position = ifelse((i == 12), "bottom", "none")) +
    theme(plot.title = element_text(colour = ifelse(i == 5, black_color, "white"))) +
    guides(color = guide_legend(order = 1),
           size = guide_legend(order = 2)) +
    plot_aesthetics

  ### Plot: Cumulative Squared Error Differences (CSSED)
  # Load Data
  df <- mean_se %>%
    as_tibble(rownames = "row") %>%
    mutate(row = as.numeric(row)) %>%
    mutate(across(-1, ~PHM - .)) %>%
    mutate(across(-1, cumsum)) %>%
    dplyr::select(-PHM, -STPHM) %>%
    pivot_longer(!row, names_to = "Model", values_to = "value") %>%
    mutate("Method" = ifelse(Model == "STSC", "STSC", "Benchmark Methods")) %>%
    mutate(Method = factor(Method, levels = c("STSC", "Benchmark Methods")))

  # Plot CSSED
  plot_cssed <- df %>%
    ggplot() +
    geom_line(aes(row,
                  value,
                  group = Model,
                  color = Method,
                  linetype = Method,
                  linewidth = Method,
                  alpha = Method)) +
    labs(title = "Cumulative Squared Error Differences",
         y = "",
         x = "") +
    scale_linetype_manual(values = c(1, 2)) +
    scale_linewidth_manual(values = c(0.75, 0.5), guide = "none") +
    scale_alpha_manual(values = c(1, 0.5), guide = "none")  +
    scale_color_manual(values = c(black_color, grey_color)) +
    scale_y_continuous(expand = y_expand) +
    theme(plot.title = element_text(colour = ifelse(i == 5, black_color, "white"))) +
    theme(legend.position = ifelse((i == 12), "bottom", "none")) +
    guides(color = guide_legend(override.aes = list(size = 14, alpha = 1, linewidth = 1.25))) + 
    geom_rect(data = shaded_areas,
              aes(xmin = start,
                  xmax = end,
                  ymin = -Inf,
                  ymax = Inf),
              fill = lightgrey_color,
              color = black_color,
              show.legend = FALSE) +
    plot_aesthetics

  ### Plot: Cumulative CRPS Differences
  # Load Data
  df <- mean_crps %>%
    as_tibble(rownames = "row") %>%
    mutate(row = as.numeric(row)) %>%
    mutate(across(-1, ~STPHM - .)) %>%
    mutate(across(-1, cumsum)) %>%
    dplyr::select(-PHM, -STPHM, -DART, -ReLa, -TRF, -XGB, -TCSR) %>%
    pivot_longer(!row, names_to = "Model", values_to = "value") %>%
    mutate("Method" = ifelse(Model == "STSC", "STSC", "Benchmark Methods")) %>%
    mutate(Method = factor(Method, levels = c("STSC", "Benchmark Methods")))

  # Plot CRPS
  plot_crps <- df %>%
    ggplot() +
    geom_line(aes(row, value, group = Model, color = Method, linetype = Method, linewidth = Method, alpha = Method)) +
    labs(title = "Cumulative CRPS Differences", y = "", x = "") +
    scale_linetype_manual(values = c(1, 2)) +
    scale_linewidth_manual(values = c(0.75, 0.5), guide = "none") +
    scale_alpha_manual(values = c(1, 0.5), guide = "none")  +
    scale_color_manual(values = c(black_color, grey_color)) +
    scale_y_continuous(expand = y_expand) +
    theme(plot.title = element_text(colour = ifelse(i == 5, black_color, "white"))) +
    theme(legend.position = ifelse((i == 12), "bottom", "none")) +
    guides(color = guide_legend(override.aes = list(size = 14, alpha = 1, linewidth = 1.25))) +
    geom_rect(data = shaded_areas,
              aes(xmin = start,
                  xmax = end,
                  ymin = -Inf,
                  ymax = Inf),
              fill = lightgrey_color,
              color = black_color,
              show.legend = FALSE) +
    plot_aesthetics

  # Save
  write_rds(plot_dgp, glue("{path}/Results/SIM/Plots/plot_dgp_{i}.rds"))
  write_rds(plot_cssed, glue("{path}/Results/SIM/Plots/plot_cssed_{i}.rds"))
  write_rds(plot_crps, glue("{path}/Results/SIM/Plots/plot_crps_{i}.rds"))
}
####### ------------------------------------------------

In [ ]:
####### ------------------------------------------------
### DGPs 1 - 4: Combine Plots
# Load Plots
list_theta <- map(1:4, ~ read_rds(glue("{path}/Results/SIM/Plots/plot_theta_{.}.rds")))
list_cssed <- map(1:4, ~ read_rds(glue("{path}/Results/SIM/Plots/plot_cssed_{.}.rds")))
list_crps  <- map(1:4, ~ read_rds(glue("{path}/Results/SIM/Plots/plot_crps_{.}.rds")))

# Combine rows
plt1 <- plot_grid(plotlist = list_theta, nrow = length(list_theta), align = "v", rel_heights = c(rep(1, length(list_theta)-1), 1.1))
plt3 <- plot_grid(plotlist = list_cssed, nrow = length(list_cssed), align = "v", rel_heights = c(rep(1, length(list_cssed)-1), 1.1))
plt2 <- plot_grid(plotlist = list_crps, nrow = length(list_crps), align = "v", rel_heights = c(rep(1, length(list_crps)-1), 1.1))

# Combine Columns
cplot_tvc <- plot_grid(plt1, plt2, plt3, ncol = 3, align = "h")

### DGPs 5 - 12: Combine Plots
# Load Plots
list_dgp   <- map(5:12, ~ read_rds(glue("{path}/Results/SIM/Plots/plot_dgp_{.}.rds")))
list_cssed <- map(5:12, ~ read_rds(glue("{path}/Results/SIM/Plots/plot_cssed_{.}.rds")))
list_crps  <- map(5:12, ~ read_rds(glue("{path}/Results/SIM/Plots/plot_crps_{.}.rds")))

# Combine rows
plt1 <- plot_grid(plotlist = list_dgp, nrow = length(list_dgp), align = "v", rel_heights = c(rep(1, length(list_dgp)-1), 1.1))
plt3 <- plot_grid(plotlist = list_cssed, nrow = length(list_cssed), align = "v", rel_heights = c(rep(1, length(list_cssed)-1), 1.1))
plt2 <- plot_grid(plotlist = list_crps, nrow = length(list_crps), align = "v", rel_heights = c(rep(1, length(list_crps)-1), 1.1))

# Combine Columns
cplot_nl <- plot_grid(plt1, plt2, plt3, ncol = 3, align = "h")

### Save
ggsave(glue("plot_sim_{snr}_tvc.pdf"),
       cplot_tvc,
       path = glue("{path}/Results/SIM/Plots/"),
       width = 25,
       height = 25)
ggsave(glue("plot_sim_{snr}_nl.pdf"),
       cplot_nl,
       path = glue("{path}/Results/SIM/Plots/"),
       width = 25,
       height = 37.5)

# Show Plots
options(repr.plot.width = 25, repr.plot.height = 25)
cplot_tvc
options(repr.plot.width = 25, repr.plot.height = 45)
cplot_nl
####### ------------------------------------------------

## 5) LaTex Tables

In [ ]:
####### ------------------------------------------------
### LaTeX Table -- Accuracy ###
# Get all File-Paths
snr <- "SNR05"
files <- list.files(glue("{path}/Results/SIM/Accuracy/{snr}"),
                    full.names = TRUE,
                    pattern = "res_")

# Calculate relative Average-MSFE
res_mse <- purrr::map(files, ~ read_rds(.x)) %>%
  purrr::map(~.x$mse) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  purrr::reduce(`+`) %>%
  divide_by(length(files)) %>%
  divide_by(.$STSC) %>%
  round(2) %>%
  dplyr::select(-STPHM, -PHM) %>%
  mutate_if(is.numeric, format, digits = 4, nsmall = 2)

# Calculate relative Average-CRPS
res_crps <- purrr::map(files, ~ read_rds(.x)) %>%
  purrr::map(~.x$mcrps) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  purrr::reduce(`+`) %>%
  divide_by(length(files)) %>%
  divide_by(.$STSC) %>%
  round(2) %>%
  dplyr::select(-STPHM, -PHM) %>%
  mutate_if(is.numeric, format, digits = 4, nsmall = 2)

# Split DGP: 1-4 -> Appendix
res_mse1 <- res_mse %>%
  dplyr::slice(1:4) %>%
  `rownames<-`(paste0("DGP ", seq_along(rownames(.)))) %>%
  rownames_to_column(" ")
res_crps1 <- res_crps %>%
  dplyr::slice(1:4) %>%
  `rownames<-`(paste0("DGP ", seq_along(rownames(.)))) %>%
  rownames_to_column(" ")

# Split DGP: 5-12 -> main script
res_mse2 <- res_mse %>%
  dplyr::slice(5:12) %>%
  `rownames<-`(paste0("DGP ", seq_along(rownames(.)))) %>%
  rownames_to_column(" ")
res_crps2 <- res_crps %>%
  dplyr::slice(5:12) %>%
  `rownames<-`(paste0("DGP ", seq_along(rownames(.)))) %>%
  rownames_to_column(" ")

# Latex Table (Appendix)
res_mse1 %>%
  bind_rows(res_crps1) %>%
  rename("STSC\\textsubscript{ExF}" = STSC_EXF,
         "STSC\\textsubscript{S10}" = STSCS10,
         "STSC\\textsubscript{SFlex}" = STSCSFLEX,
         "PCDMA\\textsubscript{5}" = PC5DMA,
         "PCDMA\\textsubscript{10}" = PC10DMA,
         "PCDMA\\textsubscript{15}" = PC15DMA) %>%
  mutate(across(everything(), ~na_if(.x, "NA")), across(everything(), ~replace_na(.x, "--"))) %>%
  kableExtra::kbl(booktabs = TRUE,
                  caption = glue("Forecast evaluation for synthetic data in terms of MSFE and ACRPS -- {snr}"),
                  format = "latex",
                  escape = FALSE,
                  digits = 2) %>%
  kableExtra::kable_styling(latex_options = c("scale_down")) %>%
  kableExtra::pack_rows("$\\overline{\\\\text{MSFE}}$", 1, 4, underline = TRUE, escape = F) %>%
  kableExtra::pack_rows("$\\overline{\\\\text{ACRPS}}$", 5, 8, underline = TRUE, escape = F) %>%
  kableExtra::footnote(general = glue("The table reports averages of the MSFE and ACRPS relative to STSC over $100$ simulation runs. Values greater than one indicate less accurate forecasts than STSC. The results are based on $380$ OOS forecasts."),
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "") %>%
  kableExtra::row_spec(4, hline_after = TRUE)

# Latex Table (main script)
res_mse2 %>%
  bind_rows(res_crps2) %>%
  rename("STSC\\textsubscript{ExF}" = STSC_EXF,
         "STSC\\textsubscript{S10}" = STSCS10,
         "STSC\\textsubscript{SFlex}" = STSCSFLEX,
         "PCDMA\\textsubscript{5}" = PC5DMA,
         "PCDMA\\textsubscript{10}" = PC10DMA,
         "PCDMA\\textsubscript{15}" = PC15DMA) %>%
  mutate(across(everything(), ~na_if(.x, "NA")), across(everything(), ~replace_na(.x, "--"))) %>%
  kableExtra::kbl(booktabs = TRUE,
                  caption = glue("Forecast evaluation for synthetic data in terms of MSFE and ACRPS -- {snr}"),
                  format = "latex",
                  escape = FALSE,
                  digits = 2) %>%
  kableExtra::kable_styling(latex_options = c("scale_down")) %>%
  kableExtra::pack_rows("$\\overline{\\\\text{MSFE}}$", 1, 8, underline = TRUE, escape = F) %>%
  kableExtra::pack_rows("$\\overline{\\\\text{ACRPS}}$", 9, 16, underline = TRUE, escape = F) %>%
  kableExtra::footnote(general = glue("The table reports averages of the MSFE and ACRPS relative to STSC over $100$ simulation runs. Values greater than one indicate less accurate forecasts than STSC. The results are based on $380$ OOS forecasts."),
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "") %>%
  kableExtra::row_spec(8, hline_after = TRUE)
####### ------------------------------------------------

In [ ]:
####### ------------------------------------------------
### LaTeX Table -- Computation Time ###
### Get Time in Seconds
sec <- read_rds(glue("{path}/Results/SIM/Timing/timing_results.rds")) %>%
  as_tibble() %>%
  dplyr::select(STSC) %>%
  round(1) %>%
  pull()

### Relative Timing Results
read_rds(glue("{path}/Results/SIM/Timing/timing_results.rds")) %>%
  as_tibble() %>%
  mutate(across(-c(T, K), ~ .x / 60)) %>%
  mutate(across(-c(T, K), ~ .x / STSC)) %>%
  rename("$T$" = T, "$K$" = K,
         "STSC\\textsubscript{S10}" = STSCS10,
         "STSC\\textsubscript{SFlex}" = STSCSFLEX,
         "PCDMA\\textsubscript{5}" = PC5DMA,
         "PCDMA\\textsubscript{10}" = PC10DMA,
         "PCDMA\\textsubscript{15}" = PC15DMA) %>%
  mutate(STSC = ifelse(row_number() == 1, paste("$\\underset{(", sec[1], " s)}{1.00}$"), STSC)) %>%
  mutate(STSC = ifelse(row_number() == 2, paste("$\\underset{(", sec[2], " s)}{1.00}$"), STSC)) %>%
  kableExtra::kbl(booktabs = TRUE,
                  caption = "Forecast evaluation for synthetic data in terms of computing time",
                  format = "latex",
                  escape = F,
                  digits = 2,
                  format.args = list(big.mark = ',')) %>%
  kableExtra::kable_styling(position = "center", latex_options = c("scale_down")) %>%
  kableExtra::footnote(general = "The table shows the computation time required (relative to STSC) to produce $380$ OOS forecasts with $T$ observations and $K$ signals. The benchmark methods are fitted using a rolling window. The results are based on SNR$_{0.5}$ and are averaged over five repetitions. For STSC, the computing time is given in seconds.",
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "") %>%
  kableExtra::column_spec(2, border_right = TRUE)
####### ------------------------------------------------

---